In [1]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tracemalloc import stop
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string
import nltk
import emoji
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import string

In [3]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
sw=nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# df = pd.read_csv("New_Dropped_Uscomments.csv",error_bad_lines=False)
df = pd.read_csv("New_DF_Positive_Negative_Netural_dropna.csv",error_bad_lines=False)
# df = pd.read_csv("Random_same_sample_ALL_3_Category.csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.head()


,video_id,comment_text,likes,replies,Polarity,Target
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,trending 😉,3,0,0.0,Neutral


In [7]:
len(df)

471631

In [8]:
df['Target'].value_counts()

Positive    203102
Neutral     195920
Negative     72609
Name: Target, dtype: int64

# Trying Only Positive and Negative Netural

In [9]:
from textblob import TextBlob

In [10]:
polarity = []
for comment in df['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)

In [11]:
x=[]
for i in df.Polarity:
    if i>0:
        x.append('Positive')
    elif i<0:
        x.append('Negative')
    else:
        x.append('Neutral')
        

In [12]:
df['Target'] = x

In [13]:
df.head()

,video_id,comment_text,likes,replies,Polarity,Target
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,trending 😉,3,0,0.0,Neutral


In [14]:
df['Target'].value_counts()

Positive    203102
Neutral     195920
Negative     72609
Name: Target, dtype: int64

In [83]:
tmp_positive = df[df['Target']=='Positive'].sample(10000)
tmp_negative = df[df['Target']=='Negative'].sample(10000)
tmp_netural = df[df['Target']=='Neutral'].sample(10000)

In [84]:
df1 = pd.concat([tmp_positive,tmp_negative,tmp_netural],ignore_index=True)

In [85]:
df1.head()

,video_id,comment_text,likes,replies,Polarity,Target
0,IqLi6Pn_rVo,Oh my god this is so gorgeous,0,0,0.700000,Positive
1,EmlH7yRKmSY,I wonder how many people took home the wrong d...,0,0,0.266667,Positive
2,FfRGxN2zeWU,Not that good but very good for age of 10. Cheers,0,0,0.805000,Positive
3,EbthMC6spAE,Why am I watching a video on Norm Macdonald wh...,1,0,0.550000,Positive
4,_cu4FHyCcT8,Daxid hax ur the best,0,0,1.000000,Positive


In [86]:
np.random.seed(42)  # Set a seed for reproducibility (optional)
df1['random'] = np.random.rand(len(df1))  # Add a random column
df1 = df1.sort_values(by=['random'])  # Sort the DataFrame by 'category' and 'random'
df1 = df1.drop('random', axis=1)  # Remove the 'random' column

In [87]:
df1.head(10)

,video_id,comment_text,likes,replies,Polarity,Target
2902,NCFO5eifDtg,Don't put these to shows together stick to the...,0,0,0.600000,Positive
4151,FfRGxN2zeWU,I cant live without you angelica hale,1,0,0.136364,Positive
21945,oUb-TwzDnk8,JP LamBakes,0,0,0.000000,Neutral
9187,3QtHD3-N5nI,I absolutely love riverdale and you're giving ...,0,0,0.400000,Positive
26008,5IP8jQNXDOo,wheres the 100k at,0,0,0.000000,Neutral
27240,4WNHB8vROg4,"OMG I never heard that Tom had passed,🌹1 of th...",0,0,0.000000,Neutral
2604,0pwttmAYCTw,Kygo is magic,0,0,0.500000,Positive
11883,H1YGg8nYkv0,"Sucks to be her, what's new?",1,0,-0.081818,Negative
25861,lg0ldbwZhPQ,The meat juice!!,0,0,0.000000,Neutral
4725,RsCsBqIlnHw,Casey do you subscribed to any boxes? If so wh...,0,0,0.350000,Positive


In [90]:
df1 = df1.drop(['video_id'],axis=1)

In [91]:
df1.head()

,comment_text,Polarity,Target
2902,Don't put these to shows together stick to the...,0.600000,Positive
4151,I cant live without you angelica hale,0.136364,Positive
21945,JP LamBakes,0.000000,Neutral
9187,I absolutely love riverdale and you're giving ...,0.400000,Positive
26008,wheres the 100k at,0.000000,Neutral


In [92]:
# TextBlob("").polarity

In [29]:
# ans=0
# for i in list(TextBlob(df1['comment_text'][3]).words):
#     print(i)
#     ans+=TextBlob(i).polarity
    
# ans

You
have
great
lips😊


0.8

In [93]:
df1['Target'].value_counts()

Positive    10000
Neutral     10000
Negative    10000
Name: Target, dtype: int64

In [94]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [95]:
sw = stopwords.words('english')
ps = PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [96]:
# df['tokenize'] = df['comment_text'].apply()

In [97]:
df1['comment_text'][27240]

'OMG I never heard that Tom had passed,🌹1 of the Greats when I was in HS.Very sad.🎵🎶🎶'

In [98]:
str=df1['comment_text'][27240]

In [99]:
tb = TextBlob(str)


In [100]:
tb.polarity

0.0

In [101]:
lst = tb.words

In [102]:
lst

WordList(['OMG', 'I', 'never', 'heard', 'that', 'Tom', 'had', 'passed', '🌹1', 'of', 'the', 'Greats', 'when', 'I', 'was', 'in', 'HS.Very', 'sad.🎵🎶🎶'])

In [34]:
# ans=0
# for i in lst:
#     print(i)
#     new_tb = TextBlob(i)
#     ans += new_tb.polarity
#     print(ans)
# #     print(new_tb.polarity)
# ans

In [103]:
df1.head()

,comment_text,Polarity,Target
2902,Don't put these to shows together stick to the...,0.600000,Positive
4151,I cant live without you angelica hale,0.136364,Positive
21945,JP LamBakes,0.000000,Neutral
9187,I absolutely love riverdale and you're giving ...,0.400000,Positive
26008,wheres the 100k at,0.000000,Neutral


In [104]:
df1['comment_text'] = df1['comment_text'].str.lower()

In [105]:
# df1 = df1.drop(['video_id','likes','replies'],axis=1)

In [106]:
df1.head()

,comment_text,Polarity,Target
2902,don't put these to shows together stick to the...,0.600000,Positive
4151,i cant live without you angelica hale,0.136364,Positive
21945,jp lambakes,0.000000,Neutral
9187,i absolutely love riverdale and you're giving ...,0.400000,Positive
26008,wheres the 100k at,0.000000,Neutral


In [107]:
def remove_sw_punc(str):
    new_str=[]
    for i in list(TextBlob(str).words):
        if i not in stopwords.words('english') and i not in string.punctuation:
#             print(i)
            new_str.append(i)
    text = ' '.join(new_str)
    return text

In [108]:
str="Best thing ever seen yet. 1:00 wants to play with the toy too. No  cat goes ohh..."
def tokenize(str):
    tb = TextBlob(str).words
    
    return list(tb)
    
#     print(tb)
#     print(type(tb))

In [47]:
# df1['comment_text'] = df1['comment_text'].apply(remove_sw_punc)

In [109]:
import emoji

def remove_emoji(text):
    return ''.join(char for char in text if not emoji.is_emoji(char))


In [75]:
text="Hello, 😊 how are you?"
# text='awesome💕💕🌸🌌'

In [76]:
remove_emoji(text)

'Hello,  how are you?'

In [77]:
TextBlob(remove_emoji(text)).words

WordList(['Hello', 'how', 'are', 'you'])

In [110]:
df1['comment_text'] = df1['comment_text'].apply(remove_emoji)

In [112]:
df1['comment_text'][27240]

'omg i never heard that tom had passed,1 of the greats when i was in hs.very sad.'

In [113]:
df1['comment_to_token'] = df1['comment_text'].apply(tokenize)

In [114]:
df1.head()

,comment_text,Polarity,Target,comment_to_token
2902,don't put these to shows together stick to the...,0.600000,Positive,"[do, n't, put, these, to, shows, together, sti..."
4151,i cant live without you angelica hale,0.136364,Positive,"[i, cant, live, without, you, angelica, hale]"
21945,jp lambakes,0.000000,Neutral,"[jp, lambakes]"
9187,i absolutely love riverdale and you're giving ...,0.400000,Positive,"[i, absolutely, love, riverdale, and, you, 're..."
26008,wheres the 100k at,0.000000,Neutral,"[wheres, the, 100k, at]"


In [119]:
def find_polarity_from_token(lst):
    ans=0
    for i in lst:
        ans+=TextBlob(i).polarity
    return ans

In [123]:
find_polarity_from_token(df1['comment_to_token'][9187])

1.5999999999999999

In [116]:
type(df1['comment_to_token'][2902])

list

In [139]:
# lst = df1['comment_to_token'][9187]
# ans=0
# for i in lst:
#     print(i,ans)
#     ans+=TextBlob(i).polarity
# ans

In [138]:
# TextBlob('spice').polarity

0.0

In [140]:
# st="awesome💕💕🌸🌌"
# list(TextBlob(st).words)

In [141]:
df1['New_Polarity'] = df1['comment_to_token'].apply(find_polarity_from_token)

In [142]:
df1.head()

,comment_text,Polarity,Target,comment_to_token,New_Polarity
2902,don't put these to shows together stick to the...,0.600000,Positive,"[do, n't, put, these, to, shows, together, sti...",0.600000
4151,i cant live without you angelica hale,0.136364,Positive,"[i, cant, live, without, you, angelica, hale]",0.136364
21945,jp lambakes,0.000000,Neutral,"[jp, lambakes]",0.000000
9187,i absolutely love riverdale and you're giving ...,0.400000,Positive,"[i, absolutely, love, riverdale, and, you, 're...",1.600000
26008,wheres the 100k at,0.000000,Neutral,"[wheres, the, 100k, at]",0.000000


In [146]:
# TextBlob(df1['comment_text'][9187]).polarity

In [147]:
def Create_new_target(no):
    if no>0:
        return 1
    elif no<0:
        return -1;
    else:
        return 0

In [148]:
df1['New_Target'] = df1['New_Polarity'].apply(Create_new_target)

In [149]:
df1.head()

,comment_text,Polarity,Target,comment_to_token,New_Polarity,New_Target
2902,don't put these to shows together stick to the...,0.600000,Positive,"[do, n't, put, these, to, shows, together, sti...",0.600000,1
4151,i cant live without you angelica hale,0.136364,Positive,"[i, cant, live, without, you, angelica, hale]",0.136364,1
21945,jp lambakes,0.000000,Neutral,"[jp, lambakes]",0.000000,0
9187,i absolutely love riverdale and you're giving ...,0.400000,Positive,"[i, absolutely, love, riverdale, and, you, 're...",1.600000,1
26008,wheres the 100k at,0.000000,Neutral,"[wheres, the, 100k, at]",0.000000,0


In [150]:
df1['New_Target'].value_counts()

 1    10335
 0    10115
-1     9550
Name: New_Target, dtype: int64

In [151]:
df1.to_csv("Try_Error_Comment_Data_(Random 10000 each sample).csv",index=False)

In [152]:
new_df = pd.read_csv("Try_Error_Comment_Data_(Random 10000 each sample).csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [153]:
new_df.head()

,comment_text,Polarity,Target,comment_to_token,New_Polarity,New_Target
0,don't put these to shows together stick to the...,0.600000,Positive,"['do', ""n't"", 'put', 'these', 'to', 'shows', '...",0.600000,1
1,i cant live without you angelica hale,0.136364,Positive,"['i', 'cant', 'live', 'without', 'you', 'angel...",0.136364,1
2,jp lambakes,0.000000,Neutral,"['jp', 'lambakes']",0.000000,0
3,i absolutely love riverdale and you're giving ...,0.400000,Positive,"['i', 'absolutely', 'love', 'riverdale', 'and'...",1.600000,1
4,wheres the 100k at,0.000000,Neutral,"['wheres', 'the', '100k', 'at']",0.000000,0


In [78]:
# tokenize(str)

In [97]:
# from sklearn.feature_extraction.text import CountVectorizer

In [98]:
# vectorizer = CountVectorizer()

In [99]:
# X = vectorizer.fit_transform(tb).toarray()

In [154]:
# print(X)
# X.reshape(1,len(X.shape)).shape

In [155]:
def polarity(str):
    tb = TextBlob(str)
    return tb.polarity

In [156]:
# df1['New_Target'] = df1['comment_text'].apply(polarity)

In [157]:
def lable_target_no(no):
    if no>0:
        return 1
    elif no<0:
        return -1
    else:
        return 0

In [158]:
df1.head()

,comment_text,Polarity,Target,comment_to_token,New_Polarity,New_Target
2902,don't put these to shows together stick to the...,0.600000,Positive,"[do, n't, put, these, to, shows, together, sti...",0.600000,1
4151,i cant live without you angelica hale,0.136364,Positive,"[i, cant, live, without, you, angelica, hale]",0.136364,1
21945,jp lambakes,0.000000,Neutral,"[jp, lambakes]",0.000000,0
9187,i absolutely love riverdale and you're giving ...,0.400000,Positive,"[i, absolutely, love, riverdale, and, you, 're...",1.600000,1
26008,wheres the 100k at,0.000000,Neutral,"[wheres, the, 100k, at]",0.000000,0


In [36]:
df1['Target_in_no'] = df1['New_Target'].apply(lable_target_no)

In [37]:
df1['Target_in_no'].value_counts()

 1    70000
-1    70000
 0    70000
Name: Target_in_no, dtype: int64

In [38]:
df1 = df1.drop(['video_id'],axis=1)

In [39]:
df1.head()

,comment_text,likes,replies,Polarity,Target,New_Target,Target_in_no
0,Hahaha E! News are retarded. Love you Jim!,0,0,0.025000,Positive,0.025000,1
1,the guy was sooo. cute,0,0,0.500000,Positive,0.500000,1
2,"If you want to turn it up a notch, saute some ...",0,0,1.000000,Positive,1.000000,1
3,We have Zwitsal here ini Indonesia. It was my ...,0,0,0.750000,Positive,0.750000,1
4,maybe you can't love someone until you love yo...,0,0,0.441071,Positive,0.441071,1


In [41]:
# df1['Tokenize_word'] = df1['comment_text'].apply(tokenize)

In [42]:
df1.head()

,comment_text,likes,replies,Polarity,Target,New_Target,Target_in_no
0,Hahaha E! News are retarded. Love you Jim!,0,0,0.025000,Positive,0.025000,1
1,the guy was sooo. cute,0,0,0.500000,Positive,0.500000,1
2,"If you want to turn it up a notch, saute some ...",0,0,1.000000,Positive,1.000000,1
3,We have Zwitsal here ini Indonesia. It was my ...,0,0,0.750000,Positive,0.750000,1
4,maybe you can't love someone until you love yo...,0,0,0.441071,Positive,0.441071,1


In [43]:
# df1 = df1.drop(['Tokenize_word'],axis=1)

In [44]:
# df1.head()

In [45]:
# df1['comment_text'] = df1['comment_text'].str.lower()

In [46]:
df1.head()

,comment_text,likes,replies,Polarity,Target,New_Target,Target_in_no
0,hahaha e! news are retarded. love you jim!,0,0,0.025000,Positive,0.025000,1
1,the guy was sooo. cute,0,0,0.500000,Positive,0.500000,1
2,"if you want to turn it up a notch, saute some ...",0,0,1.000000,Positive,1.000000,1
3,we have zwitsal here ini indonesia. it was my ...,0,0,0.750000,Positive,0.750000,1
4,maybe you can't love someone until you love yo...,0,0,0.441071,Positive,0.441071,1


In [48]:
# df1['Target_in_no'].value_counts()

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [50]:
countvector=CountVectorizer(ngram_range=(2,2))


In [51]:
x=countvector.fit_transform(df1['comment_text'])
y = df1['Target_in_no']

In [52]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.3,random_state=101)

In [ ]:
# model1=MultinomialNB()

In [53]:
x_train1.shape

(147000, 946655)

In [54]:
x_test1.shape

(63000, 946655)

In [ ]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=100,criterion='entropy')
randomclassifier.fit(x_train1,y_train1)

In [ ]:
y_pred = randomclassifier.predict(x_test1)